# Event set generation

In [ ]:
from joblib import Parallel, delayed
from climada.hazard import TCTracks, TropCyclone, Centroids
from climada.entity import Exposures
from climada.entity.exposures import LitPop

In [ ]:
## set parameters
# Exposure
my_country = 'aus'
m, n = 1,1 # based on analysis by Eberenz et al. (2020), (1,1) is the best combination.
my_fin_mode = 'pc'
ref_year = 2023

# Hazard
start_year = 1980
end_year = 2023
n_synth_tracks = 1000

my_res_arcsec = 600

## Exposure

In [ ]:
exp1 = LitPop.from_countries(my_country, my_res_arcsec, (m,n), my_fin_mode, reference_year = ref_year)
exp1.check()

# Remove Lord Howe Island and Macquarie Island
exp = Exposures(exp1.gdf[exp1.geometry.x <= 155])
exp.check()

## Hazard

In [ ]:
def tracks_in_exp(tracks, exposure, buffer=1.0):
    """Select only the tracks that are in the vicinity (buffer) of an exposure.

    Each exposure point/geometry is extended to a disc of radius `buffer`. Each track is
    converted to a line and extended by a radius `buffer`.

    Parameters
    ----------
    exposure : Exposure
        Exposure used to select tracks.
    buffer : float, optional
        Size of buffer around exposure geometries (in the units of `exposure.crs`),
        see `geopandas.distance`. Default: 1.0

    Returns
    -------
    filtered_tracks : TCTracks
        TCTracks object with tracks from tc_tracks intersecting the exposure whitin a buffer
        distance.
    """

    if buffer <= 0.0:
        raise ValueError(f"buffer={buffer} is invalid, must be above zero.")
    try:
        exposure.geometry
    except AttributeError:
        raise Exception("this is not an Exposures object")

    exp_buffer = exposure.gdf.buffer(distance=buffer, resolution=0)
    exp_buffer = exp_buffer.unary_union

    # 'split_lines_antimeridian=False' is used to avoid a bug in current CLIMADA / geopandas version
    tc_tracks_lines = tracks.to_geodataframe(split_lines_antimeridian=False).buffer(distance=buffer)
    select_tracks = tc_tracks_lines.intersects(exp_buffer)
    tracks_in_exp = [track for j, track in enumerate(tracks.data) if select_tracks[j]]
    filtered_tracks = TCTracks(tracks_in_exp)

    return filtered_tracks

In [ ]:
tracks = TCTracks.from_ibtracs_netcdf(year_range = (start_year, end_year))
my_tracks = tracks_in_exp(tracks, exp)

print(f'Total number of historical tracks: {tracks.size}')
print(f'Number of historical tracks in {my_country}: {my_tracks.size}')

In [ ]:
# Generate synthetic tracks
my_tracks.equal_timestep()
my_tracks.calc_perturbed_trajectories(nb_synth_tracks = n_synth_tracks, decay=False);

print(f'Number of tracks in {my_country} (inc. synthetic): {my_tracks.size:,.0f}')

In [ ]:
buffer = 1.0
lon = exp.geometry.x
lat = exp.geometry.y
polygon = (lon.min() - buffer, lat.min() - buffer, lon.max() + buffer, lat.max() + buffer)

# Centroids
cent = Centroids.from_pnt_bounds(polygon, res=my_res_arcsec/3600)
cent.set_on_land()
cent.set_region_id()

In [ ]:
def convert_tctracks_to_tropcyclone_in_batches(i, track_batch, cent):
    haz_batch = TropCyclone.from_tracks(track_batch, centroids=cent, intensity_thres=10.0)
    haz_batch.write_hdf5(f'Hazards/haz_{my_country}_{n_synth_tracks}synth_part_{i}.hdf5')

In [ ]:
batch_size = (n_synth_tracks + 1) * 1
num_batches = len(my_tracks.data) // batch_size
track_batches = [TCTracks(my_tracks.data[i * batch_size:min((i + 1) * batch_size, len(my_tracks.data))]) for i in range(num_batches)]


In [ ]:
Parallel(n_jobs=8)(
    delayed(convert_tctracks_to_tropcyclone_in_batches)(i, track_batch, cent)
    for i, track_batch in enumerate(track_batches)
);

In [ ]:
# Load all the TropCyclone batches into one object
files = [f'Hazards/haz_{my_country}_{n_synth_tracks}synth_part_{i}.hdf5' for i in range(num_batches)]
haz = TropCyclone.concat([TropCyclone.from_hdf5(file) for file in files])

In [ ]:
haz.check()

In [ ]:
# Save hazard
haz.write_hdf5(f'Hazards/haz_{my_country}_{n_synth_tracks}synth.hdf5')